# 0. import lib

In [2]:
import numpy as np
import pandas as pd
import math

# 1. generate mini data

In [4]:
data = pd.read_table('./data/20140803_train.txt', sep=',', names=['id','lat','lng','status','datetime'])
data.info

<bound method DataFrame.info of              id        lat         lng  status           datetime
0             1  30.624806  104.136604       1  2014/8/3 21:18:46
1             1  30.624809  104.136612       1  2014/8/3 21:18:15
2             1  30.624811  104.136587       1  2014/8/3 21:20:17
3             1  30.624811  104.136596       1  2014/8/3 21:19:16
4             1  30.624811  104.136619       1  2014/8/3 21:17:44
...         ...        ...         ...     ...                ...
53045402  13605  30.732042  103.998304       1  2014/8/3 11:24:41
53045403  13605  30.732262  103.997677       1  2014/8/3 11:28:58
53045404  13605  30.732322  103.997589       1  2014/8/3 11:25:21
53045405  13605  30.732406  103.997811       1  2014/8/3 11:25:12
53045406  13605  30.732416  103.997727       1  2014/8/3 11:25:15

[53045407 rows x 5 columns]>

In [5]:
# np.set_printoptions(suppress=True)
# pd.set_option('display.float_format', lambda x:'%.3f'%x) 
data.describe()

,id,lat,lng,status
count,53045407.000,53045407.000,53045407.000,53045407.000
mean,6878.455,30.652,104.060,0.564
std,3907.969,0.043,0.052,0.496
min,1.000,30.291,103.270,0.000
25%,3545.000,30.629,104.032,0.000
50%,6931.000,30.652,104.064,1.000
75%,10248.000,30.677,104.092,1.000
max,13605.000,31.032,104.610,1.000


In [7]:
# select id: top 100 and save
mini_data = data[data['id'] < 100]
mini_data.describe()
mini_data.to_csv('./data/mini_data.csv', index=False)

In [8]:

mini_data['id'].value_counts().sort_values(ascending=False)

16    6457
92    6341
75    6334
54    6278
50    6222
      ... 
79    1028
48     966
63     697
64     430
23     393
Name: id, Length: 99, dtype: int64

# 2. spilt gird


In [10]:
# read mini data
data = pd.read_csv('./data/mini_data.csv')
data.describe()

,id,lat,lng,status
count,359793.000,359793.000,359793.000,359793.000
mean,52.747,30.655,104.058,0.546
std,28.204,0.047,0.060,0.498
min,1.000,30.323,103.497,0.000
25%,31.000,30.629,104.033,0.000
50%,55.000,30.654,104.066,1.000
75%,76.000,30.678,104.094,1.000
max,99.000,30.996,104.276,1.000


In [11]:
# split gird

LAT1, LAT2 = 30.3226, 30.9957
LNG1, LNG2 = 103.4970, 104.2757

column_num, row_num = 200, 200
gird_lat_span = (LAT2 - LAT1)/row_num
gird_lng_span = (LNG2 - LNG1)/column_num

def LngLat2GirdID(lat,lng):
    if lng < LNG1 or lng > LNG2 or lat < LAT1 or lat > LAT2:
        return -1

    return int((lat-LAT1)/gird_lat_span) + int((lng-LNG1)/gird_lng_span) * column_num 


def GirdID2LngLat(gird_id):
    curr_row = int(gird_id/row_num)
    curr_column = gird_id % column_num

    return [LAT1 + gird_lat_span * (curr_row + 0.5 ), LNG1 + gird_lng_span * (curr_column + 0.5)]


In [12]:
data['gird_id'] = data.apply(lambda x: LngLat2GirdID(x['lat'], x['lng']), axis=1)
data.describe()

,id,lat,lng,status,gird_id
count,359793.000,359793.000,359793.000,359793.000,359793.000
mean,52.747,30.655,104.058,0.546,28834.507
std,28.204,0.047,0.060,0.498,3094.431
min,1.000,30.323,103.497,0.000,176.000
25%,31.000,30.629,104.033,0.000,27497.000
50%,55.000,30.654,104.066,1.000,29286.000
75%,76.000,30.678,104.094,1.000,30692.000
max,99.000,30.996,104.276,1.000,39870.000


In [13]:
data['datetime'] = pd.to_datetime(data['datetime'])
data_hour_id = data.groupby([data['id'], data['datetime'].dt.hour])
data_hour_id.count()

lat  lng  status  datetime  gird_id
id datetime                                     
1  6          59   59      59        59       59
   7         109  109     109       109      109
   8         118  118     118       118      118
   9         142  142     142       142      142
   10        120  120     120       120      120
...          ...  ...     ...       ...      ...
99 19        135  135     135       135      135
   20        136  136     136       136      136
   21        159  159     159       159      159
   22        162  162     162       162      162
   23        151  151     151       151      151

[1718 rows x 5 columns]

# 3. time slice data

In [14]:
from collections import UserDict
result = UserDict()

for i in range(24):
    result[i] = []

result

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: []}

In [15]:
for (id, hour), h_data in data_hour_id:
    gird_data = h_data['gird_id'].to_list()
    paths = ['%d-%d' % (gird_data[i],gird_data[i+1]) for i in range(len(gird_data)-1)]
    paths = sorted(set(paths), key=paths.index)
    result[hour] += paths
    # print(id, hour, paths)

result[6]

['32098-32098',
 '32484-32484',
 '32484-32684',
 '32684-32685',
 '32685-32687',
 '32687-31887',
 '31887-32488',
 '32488-31888',
 '31888-31888',
 '31888-32288',
 '32288-32088',
 '32088-31888',
 '31888-32088',
 '32088-32089',
 '32089-31891',
 '31891-29692',
 '29692-29292',
 '29292-29292',
 '29292-31692',
 '31692-29692',
 '29692-29692',
 '29692-29893',
 '29893-30093',
 '30093-31693',
 '31693-29293',
 '29293-30293',
 '30293-30293',
 '30293-31293',
 '31293-31293',
 '31293-30494',
 '30494-30494',
 '30494-31894',
 '31894-31094',
 '31094-30895',
 '30895-32095',
 '32095-30695',
 '30695-30695',
 '30695-31495',
 '31495-30696',
 '30696-30696',
 '30696-32296',
 '32296-30697',
 '30697-30497',
 '30497-31897',
 '31897-32097',
 '32097-30498',
 '30498-30498',
 '25891-25891',
 '25891-26091',
 '26091-25891',
 '25891-26092',
 '26092-25892',
 '25892-26092',
 '25892-26292',
 '26292-25892',
 '25892-25892',
 '25892-25692',
 '25692-26292',
 '25692-25692',
 '25692-25892',
 '26292-25692',
 '26292-26492',
 '26492-

In [63]:
# remove repeat
# for i in range(24):
#     result[i] = sorted(set(result[i]), key=result[i].index)

# 4. DTM

In [16]:
# DTM
import logging
import os
from gensim import corpora, utils
from gensim.models.wrappers.dtmmodel import DtmModel
import numpy as np

In [17]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [18]:
documents = []
for i in range(6,24):
    documents.append(result[i])

documents

[['32098-32098',
  '32484-32484',
  '32484-32684',
  '32684-32685',
  '32685-32687',
  '32687-31887',
  '31887-32488',
  '32488-31888',
  '31888-31888',
  '31888-32288',
  '32288-32088',
  '32088-31888',
  '31888-32088',
  '32088-32089',
  '32089-31891',
  '31891-29692',
  '29692-29292',
  '29292-29292',
  '29292-31692',
  '31692-29692',
  '29692-29692',
  '29692-29893',
  '29893-30093',
  '30093-31693',
  '31693-29293',
  '29293-30293',
  '30293-30293',
  '30293-31293',
  '31293-31293',
  '31293-30494',
  '30494-30494',
  '30494-31894',
  '31894-31094',
  '31094-30895',
  '30895-32095',
  '32095-30695',
  '30695-30695',
  '30695-31495',
  '31495-30696',
  '30696-30696',
  '30696-32296',
  '32296-30697',
  '30697-30497',
  '30497-31897',
  '31897-32097',
  '32097-30498',
  '30498-30498',
  '25891-25891',
  '25891-26091',
  '26091-25891',
  '25891-26092',
  '26092-25892',
  '25892-26092',
  '25892-26292',
  '26292-25892',
  '25892-25892',
  '25892-25692',
  '25692-26292',
  '25692-25692

In [19]:
class DTMcorpus(corpora.textcorpus.TextCorpus):

    def get_texts(self):
        return self.input

    def __len__(self):
        return len(self.input)

corpus = DTMcorpus(documents)

INFO:gensim.corpora.textcorpus:Initializing dictionary
INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(32230 unique tokens: ['17913-17913', '17913-18113', '18113-17913', '18113-18113', '18311-18311']...) from 18 documents (total 132016 corpus positions)


In [20]:
model = DtmModel('./utils/dtm-win64.exe', corpus,[1]*len(documents), num_topics=5,
                 id2word=corpus.dictionary, initialize_lda=True)

INFO:gensim.models.wrappers.dtmmodel:serializing temporary corpus to C:\Users\laulliam\AppData\Local\Temp\34f0f6_train-mult.dat
INFO:gensim.corpora.bleicorpus:no word id mapping provided; initializing from corpus
INFO:gensim.corpora.bleicorpus:storing corpus in Blei's LDA-C format into C:\Users\laulliam\AppData\Local\Temp\34f0f6_train-mult.dat
DEBUG:smart_open.smart_open_lib:{'uri': 'C:\\Users\\laulliam\\AppData\\Local\\Temp\\34f0f6_train-mult.dat', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.corpora.bleicorpus:saving vocabulary of 32230 words to C:\Users\laulliam\AppData\Local\Temp\34f0f6_train-mult.dat.vocab
DEBUG:smart_open.smart_open_lib:{'uri': 'C:\\Users\\laulliam\\AppData\\Local\\Temp\\34f0f6_train-mult.dat.vocab', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'comp

In [21]:
for j in range(18):
    for i in range(0, 5):
        print ("Distribution of Topic %d %f in timeslice %d" % (i, model.gamma_[j, i], j))

Distribution of Topic 0 0.199593 in timeslice 0
Distribution of Topic 1 0.199028 in timeslice 0
Distribution of Topic 2 0.200867 in timeslice 0
Distribution of Topic 3 0.200919 in timeslice 0
Distribution of Topic 4 0.199593 in timeslice 0
Distribution of Topic 0 0.200802 in timeslice 1
Distribution of Topic 1 0.199553 in timeslice 1
Distribution of Topic 2 0.199012 in timeslice 1
Distribution of Topic 3 0.200644 in timeslice 1
Distribution of Topic 4 0.199988 in timeslice 1
Distribution of Topic 0 0.201103 in timeslice 2
Distribution of Topic 1 0.200500 in timeslice 2
Distribution of Topic 2 0.199788 in timeslice 2
Distribution of Topic 3 0.199041 in timeslice 2
Distribution of Topic 4 0.199568 in timeslice 2
Distribution of Topic 0 0.200882 in timeslice 3
Distribution of Topic 1 0.198163 in timeslice 3
Distribution of Topic 2 0.199861 in timeslice 3
Distribution of Topic 3 0.200529 in timeslice 3
Distribution of Topic 4 0.200565 in timeslice 3
Distribution of Topic 0 0.199850 in time

In [24]:
time_slice_num = 17

model.show_topic(topicid=0, time=time_slice_num, topn=200)

# topic_0 = [ w[1].split('-') for w in model.show_topic(topicid=0, time=time_slice_num, topn=200)]
# topic_1 = [ w[1].split('-') for w in model.show_topic(topicid=1, time=time_slice_num, topn=200)]
# topic_2 = [ w[1].split('-') for w in model.show_topic(topicid=2, time=time_slice_num, topn=200)]
# topic_3 = [ w[1].split('-') for w in model.show_topic(topicid=3, time=time_slice_num, topn=200)]
# topic_4 = [ w[1].split('-') for w in model.show_topic(topicid=4, time=time_slice_num, topn=200)]

# topic_0 = pd.DataFrame(topic_0, columns=['source', 'target'])
# topic_1 = pd.DataFrame(topic_1, columns=['source', 'target'])
# topic_2 = pd.DataFrame(topic_2, columns=['source', 'target'])
# topic_3 = pd.DataFrame(topic_3, columns=['source', 'target'])
# topic_4 = pd.DataFrame(topic_4, columns=['source', 'target'])

# topic_0['source'] = topic_0.apply(lambda x: GirdID2LngLat(int(x['source'])), axis=1)
# topic_0['target'] = topic_0.apply(lambda x: GirdID2LngLat(int(x['target'])), axis=1)
# topic_1['source'] = topic_1.apply(lambda x: GirdID2LngLat(int(x['source'])), axis=1)
# topic_1['target'] = topic_1.apply(lambda x: GirdID2LngLat(int(x['target'])), axis=1)
# topic_2['source'] = topic_2.apply(lambda x: GirdID2LngLat(int(x['source'])), axis=1)
# topic_2['target'] = topic_2.apply(lambda x: GirdID2LngLat(int(x['target'])), axis=1)
# topic_3['source'] = topic_3.apply(lambda x: GirdID2LngLat(int(x['source'])), axis=1)
# topic_3['target'] = topic_3.apply(lambda x: GirdID2LngLat(int(x['target'])), axis=1)
# topic_4['source'] = topic_4.apply(lambda x: GirdID2LngLat(int(x['source'])), axis=1)
# topic_4['target'] = topic_4.apply(lambda x: GirdID2LngLat(int(x['target'])), axis=1)


[(0.0012502869421418348, '28896-29097'),
 (0.0012380387341452232, '28908-28908'),
 (0.001207704204743416, '29288-28888'),
 (0.0011504779729255706, '29310-29310'),
 (0.001130201154945906, '29288-29288'),
 (0.0010883580773946035, '31703-31703'),
 (0.0010466984855151444, '29499-29499'),
 (0.0010101105725370926, '28896-28896'),
 (0.00100338280801524, '29688-29288'),
 (0.0009353223124901898, '29696-29696'),
 (0.0009275426985023041, '28899-28899'),
 (0.0009173006632119209, '28700-28700'),
 (0.0008719451363108295, '29298-29298'),
 (0.0008405480239695512, '29296-29296'),
 (0.0008121173593176773, '29498-29498'),
 (0.0008037805082704451, '29897-29897'),
 (0.0008032346278235811, '29688-29688'),
 (0.0007790889459147539, '29292-29292'),
 (0.0007733754131073903, '29692-29692'),
 (0.0007637797453760816, '30099-30099'),
 (0.0007616043162404052, '29099-29099'),
 (0.0007410392977775071, '31697-31697'),
 (0.0007230701317327513, '28696-28696'),
 (0.0007221329380788395, '30890-30890'),
 (0.0007074461891356